<a href="https://colab.research.google.com/github/felipmorais/Affect-and-Behaviors-Dynamics-Automatic-Calculation---PhD-Dissertation/blob/main/Affect_and_Behaviors_Dynamics_Automatic_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analysis of the dynamics between emotions and behaviors in the Brazilian context
---


This code was developed to calculate the dynamics between affect and behaviors. It computes three different types of calculations: The first one calculates the probability of an emotion to be the reason for the onset of a behavior. The second one calculates the opposite, i.e., the probability of a behavior to be the reason for the onset of an emotion. And, the third one calculates the co-occurrence probability, i.e., the probability of a given behavior and a given emotion to occurr at the same time. Unlike other open-source code, this code is designed to:

1. Calculate the adjusted L metric;
2. Apply a t-test to calculate the statistical significance of the results considering all students;
3. Apply the p-value adjustment calculation for multiple tests, considering the Benjamini/Hochberg (BH) method;
4. Print calculated statistics in tabular form;
5. Compute the dynamics between affect and behaviors 
6. Compute the dynamics between behaviors and affect  
7. Compute the co-occurrence between behaviors and affect
* Items 5, 6, and 7 are based on calculated statistics, considering only transitions reported as significant after p-value adjustment based on BH method; Also, these items include the analysis when considering the students gender.


---
**Important:** This code is part of a scientific article submitted for evaluation.
*Please do not share this code until the article is published.*

Execution 1: Lib imports and data files load
---
This code is used to import and pre-process the emotions file. It is important that this file is ordered according to the occurrence of emotions and the id of the respective student. It also loads the file with the emotions permanence time.


In [ ]:
# libs import and file loader
import numpy as np
import pandas as pd 
from scipy import stats
from scipy.stats import shapiro
import statsmodels.stats.multitest as multi
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
import networkx as nx
import statistics

from google.colab import drive

drive.mount('/content/drive')

# file with all logs with emotion labels
# CSV Data example:
# student_id,emotion_label,behavior_label
# 1,ENGAGEMENT,ON TASK
# 1,ENGAGEMENT,ON TASK
# 1,CONFUSION,ON TASK
# 1,CONFUSION,ON TASK
# 1,CONFUSION,ON TASK
# 1,CONFUSION,ON TASK CONVERSATION
# 1,CONFUSION,ON TASK CONVERSATION
# 1,CONFUSION,ON TASK CONVERSATION
# 1,BOREDOM,ON TASK
# 1,BOREDOM,ON TASK
# 1,BOREDOM,ON TASK
# 1,BOREDOM,ON SYSTEM
# 1,BOREDOM,ON SYSTEM
# 1,BOREDOM,ON SYSTEM
# 1,FRUSTRATION,ON TASK
# 1,FRUSTRATION,ON TASK
# 1,FRUSTRATION,ON TASK
# 1,ENGAGEMENT,ON TASK
# 1,ENGAGEMENT,ON TASK
# 1,ENGAGEMENT,ON TASK
# 1,ENGAGEMENT,ON TASK
affect_and_behavior_labels_path = "/content/drive/MyDrive/<PATH_TO_THE_AFFECT_AND_BEHAVIORS_LABELS_FILE>.csv"
df_labels = pd.read_csv(affect_and_behavior_labels_path, delimiter=",")


# students information about gender
# CSV data example:
# student_id,gender
# 1,F
# 2,F
# 3,M
# 4,M
# 5,M
# 6,F
# 7,M
students_info_path = "/content/drive/MyDrive/<PATH_TO_THE_STUDENTS_INFO_FILE>.csv"
df_students_info = pd.read_csv(students_info_path, delimiter=",")

# file with the emotion duration (permanence time in each emotion) in seconds of the students
# CSV data example:
# student_id,emotion_label,duration
# 1,ENGAGEMENT,12
# 1,OTHER,2
# 1,ENGAGEMENT,19
# 1,OTHER,2
# 1,ENGAGEMENT,12
# 1,FRUSTRATION,2
# 1,ENGAGEMENT,4
# 1,CONFUSION,2
# 1,ENGAGEMENT,34
# 1,OTHER,15
# 1,ENGAGEMENT,13
# 1,CONFUSION,17
# 1,FRUSTRATION,3
emotions_duration_path = "/content/drive/MyDrive/<PATH_TO_THE_EMOTIONS_DURAION_FILE>.csv"
df_emotions_duration = pd.read_csv(emotions_duration_path, delimiter=",")

# data preparation for the log files
df_labels.loc[df_labels.emotion_label == "FRUSTRATION", "emotion_label"] = "FRU"
df_labels.loc[df_labels.emotion_label == "ENGAGEMENT", "emotion_label"] = "ENG"
df_labels.loc[df_labels.emotion_label == "CONFUSION", "emotion_label"] = "CON"
df_labels.loc[df_labels.emotion_label == "BOREDOM", "emotion_label"] = "BOR"
df_labels.loc[df_labels.emotion_label == "OTHER", "emotion_label"] = "OTH"
# remove na lines from the null emotion labels 
df_labels = df_labels[df_labels.emotion_label.notna()]

# data preparation for the log files
df_labels.loc[df_labels.behavior_label == "ON TASK", "behavior_label"] = "ON_TASK"
df_labels.loc[df_labels.behavior_label == "ON SYSTEM", "behavior_label"] = "ON_SYSTEM"
df_labels.loc[df_labels.behavior_label == "ON TASK RESOURCE", "behavior_label"] = "ON_TASK_RES"
df_labels.loc[df_labels.behavior_label == "OFF TASK", "behavior_label"] = "OFF_TASK"
df_labels.loc[df_labels.behavior_label == "ON TASK CONVERSATION", "behavior_label"] = "ON_TASK_CONV"
# remove na lines from the null emotion labels 
df_labels = df_labels[df_labels.behavior_label.notna()]

# data preparation for the emotion duration file
df_emotions_duration.loc[df_emotions_duration.emotion_label == "FRUSTRATION", "emotion_label"] = "FRU"
df_emotions_duration.loc[df_emotions_duration.emotion_label == "ENGAGEMENT", "emotion_label"] = "ENG"
df_emotions_duration.loc[df_emotions_duration.emotion_label == "CONFUSION", "emotion_label"] = "CON"
df_emotions_duration.loc[df_emotions_duration.emotion_label == "BOREDOM", "emotion_label"] = "BOR"
df_emotions_duration.loc[df_emotions_duration.emotion_label == "OTHER", "emotion_label"] = "OTH"
# remove na lines from the null emotion labels 
df_emotions_duration = df_emotions_duration[df_emotions_duration.emotion_label.notna()]

# define which emotions will be considered and in which order
emotions_list = ['ENG', 'CON', 'FRU', 'BOR', 'OTH']
behaviors_list = ['ON_TASK', 'ON_SYSTEM', 'ON_TASK_RES', 'OFF_TASK', 'ON_TASK_CONV']
emotions_behaviors_list = emotions_list + behaviors_list

# define the metrics that will be visible when the resulting table is printed
table_metrics_to_print = ['L_metric','p_value','transition_count','multiple_test_significance']

# define the at chance level when the emotions' co-occurence is removed from the data
# this value was defined according to the paper "A Re-Analysis and Synthesis of Data on Affect Dynamics in Learning", by Shamya Karumbaiah, Ryan Baker, Jaclyn Ocumpaugh, and Alexandra Andres
# it follows the formula: 1 / (n - 1)^2, in which n is the number of considered emotions. In this case, we have considered 5 emotions: 'ENG', 'CON', 'FRU', 'BOR', 'OTH'
L_metric_at_chance_threshold=0.0625


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Execution 2: L metric calculation methods
---
This first code is imported from the article "Adjusting the L Statistic when Self-Transitions are Excluded in Affect Dynamics", by authors Jeffrey Matayoshi and Shamya Karumbaiah, 2020.

The code has been kept completely for simplicity. Howeber, we have changed some methods in order to deal with the additional information about permanence time and to bring the list of all L-values to the t-test application, for the p-value calculation.

In [ ]:
# code from https://github.com/jmatayoshi/affect-transitions

import numpy as np

def remove_self_transitions(seq):
  return [st for i, st in enumerate(seq) if i == 0 or st != seq[i - 1]]

def remove_behavior_self_transitions(seq):
  return [st for i, st in enumerate(seq) if i == 0 or st != seq[i - 1]]

def generate_sequence(seq_length, state_dict, base_rates,
                      include_self_transitions=True):
    state_ind = np.arange(len(state_dict))
    if include_self_transitions:
        seq = list(
            np.random.choice(
                list(state_dict.values()),
                size=seq_length,
                p=base_rates
            )
        )
    else:
        base_rates = np.array(base_rates)
        # Construct mapping from each state to its index in state_dict
        inv_state_dict = {}
        for i in range(len(state_dict)):
            inv_state_dict[state_dict[i]] = i
        # Sample first state
        seq = [state_dict[np.random.multinomial(1, base_rates).argmax()]] 
        for i in range(seq_length - 1):
            # Get index of previous state
            prev_ind = inv_state_dict[seq[-1]]
            # Get indices of possible next states
            next_ind = np.setdiff1d(state_ind, [prev_ind])
            # Construct conditional rates of next states
            cond_next_rates = base_rates[next_ind] / (1 - base_rates[prev_ind])
            # Sample next state
            # np.random.multinomial is faster here than np.random.choice
            seq.append(state_dict[next_ind[
                np.random.multinomial(1, cond_next_rates).argmax()]])
    return seq


def get_counts(seq, states):
    next_count = {a: 0 for a in states}
    cond_count = {a: {b: 0 for b in states} for a in states}
    num_tr = len(seq) - 1
    # Compute next and conditional counts
    for i in np.arange(1, len(seq)):
        for a in states:
            if seq[i - 1] == a:
                for b in states:
                    if seq[i] == b:
                        cond_count[a][b] += 1
                        next_count[b] += 1
                        break
                break

    cond_count_list = []
    for a in cond_count:
        cond_count_list.extend(list(cond_count[a].values()))

    return list(next_count.values()), cond_count_list


def get_affect_behaviors_counts(seq, states):
    next_count = {a: 0 for a in states}
    cond_count = {a: {b: 0 for b in states} for a in states}
    num_tr = len(seq) - 1
    # Compute next and conditional counts
    for i in np.arange(1, len(seq)):
        for a in states:
            if seq[i - 1][0] == a:
                for b in states:
                    if seq[i][1] == b:
                        cond_count[a][b] += 1
                        next_count[b] += 1
                        break
            if seq[i - 1][1] == a:
                for b in states:
                    if seq[i][0] == b:
                        cond_count[a][b] += 1
                        next_count[b] += 1
                        break
            

    cond_count_list = []
    for a in cond_count:
        cond_count_list.extend(list(cond_count[a].values()))

    return list(next_count.values()), cond_count_list

def get_affect_behaviors_cooccurence_counts(seq, states):
    next_count = {a: 0 for a in states}
    cond_count = {a: {b: 0 for b in states} for a in states}
    num_tr = len(seq) - 1
    # Compute next and conditional counts
    for i in np.arange(0, len(seq)):
        for a in states:
            # ['ENG', 'ON_TASK'], ['CON', 'ON_TASK'] 
            if seq[i][1] == a:
                for b in states:
                    if seq[i][0] == b:
                        cond_count[a][b] += 1
                        next_count[a] += 1
            
    cond_count_list = []
    for a in cond_count:
        cond_count_list.extend(list(cond_count[a].values()))

    return list(next_count.values()), cond_count_list


def get_timed_counts(original_seq, timed_seq, states):
    next_count = {a: 0 for a in states}
    cond_count = {a: {b: 0 for b in states} for a in states}
    # num_tr = len(original_seq) - 1
    # Compute next counts
    for i in np.arange(1, len(original_seq)):
        for a in states:
            if original_seq[i - 1] == a:
                for b in states:
                    if original_seq[i] == b:
                        next_count[b] += 1
                        break
                break

    # Compute conditional counts
    for i in np.arange(1, len(timed_seq)):
        for a in states:
            if timed_seq[i - 1] == a:
                for b in states:
                    if timed_seq[i] == b:
                        cond_count[a][b] += 1
                        break
                break

    cond_count_list = []
    for a in cond_count:
        cond_count_list.extend(list(cond_count[a].values()))

    return list(next_count.values()), cond_count_list


def get_L_star_vals(a, b, next_counts, cond_counts, use_mean_rates=True):
    num_states = next_counts.shape[1]
    # Column indices where next != a (i.e., transitions in T_{A_complement})
    a_comp_ind = (
        np.array([i for i in range(num_states) if i != a])
    )
    # Count transitions where prev == a and next != a
    a_comp_cond_sum = cond_counts[:, a_comp_ind + a*num_states].sum(axis=1)
    if use_mean_rates:      
        # Compute L_star using base rates averaged over the whole sample
        # of sequences; note that as opposed to the computation of
        # L_star below, we only exclude samples with P(b|a) == nan; that is,
        # we only exclude sequences with no transitions from a to another state
        sample_pos = np.flatnonzero(
            a_comp_cond_sum > 0
        )
        # Compute mean base rate of b restricted to transitions with next != a
        modified_mean_base_rate = np.mean(
            next_counts[sample_pos, b] /
            next_counts[sample_pos, :][:, a_comp_ind].sum(axis=1)
        )
        # Compute conditional rate of b restricted to transitions with next != a
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_comp_cond_sum[sample_pos]
        )
        L_star_vals = (
            (cond_rates - modified_mean_base_rate)
            / (1 - modified_mean_base_rate)
        )
    else:
        # Compute L_star using base rates from each individual sequence

        # Column indices where next != a and next != b
        a_b_comp_ind = (
            np.array([i for i in range(num_states) if i != a and i != b])
        )
        # Count transitions where next != a or next != b
        a_b_comp_sum = next_counts[:, a_b_comp_ind].sum(axis=1)
        # Count transitions where next != a        
        a_comp_sum = next_counts[:, b] + a_b_comp_sum
        # Find samples where:
        #  (a) P(b|a) != nan
        #  (b) P(b) < 1
        sample_pos = np.flatnonzero(
            (a_comp_cond_sum > 0) & (a_b_comp_sum > 0)            
        )        
        # Compute base rates of b restricted to transitions with next != a
        modified_base = (
            next_counts[sample_pos, b] / a_comp_sum[sample_pos]
        )
        # Compute conditional rate of b restricted to transitions with next != a
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_comp_cond_sum[sample_pos]
        )       
        L_star_vals = (
            (cond_rates - modified_base)
            / (1 - modified_base)
        )
    return L_star_vals

def get_L_vals(a, b, next_counts, cond_counts, use_mean_rates=True):
    num_states = next_counts.shape[1]
    # Count transitions where prev == a and next != a
    a_cond_sum = cond_counts[
        :, np.arange(num_states) + a*num_states].sum(axis=1)
    if use_mean_rates:      
        # Compute L using base rates averaged over the whole sample
        # of sequences.  Note that as opposed to the computation of
        # L below, we only exclude samples with P(b|a) == nan; that is,
        # we only exclude sequences with no transitions from a 
        sample_pos = np.flatnonzero(
            a_cond_sum > 0
        )
        # Compute mean base rate of b 
        mean_base_rate = np.mean(
            next_counts[sample_pos, b] /
            next_counts[sample_pos, :].sum(axis=1)
        )
        # Compute conditional rate of b 
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_cond_sum[sample_pos]
        )
        L_vals = (
            (cond_rates - mean_base_rate)
            / (1 - mean_base_rate)
        )
    else:
        # Compute L using base rates from each individual sequence

        # Column indices where next != a and next != b
        b_comp_ind = (
            np.array([i for i in range(num_states) if i != b])
        )
        # Count transitions where next != b
        b_comp_sum = next_counts[:, b_comp_ind].sum(axis=1)
        # Find samples where:
        #  (a) P(b|a) != nan
        #  (b) P(b) < 1
        sample_pos = np.flatnonzero(
            (a_cond_sum > 0) & (b_comp_sum > 0)            
        )        
        # Compute base rates of b
        base_rates = (
            next_counts[sample_pos, b] / next_counts[sample_pos, :].sum(axis=1)
        )
        # Compute conditional rate of b
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_cond_sum[sample_pos]
        )       
        L_vals = (
            (cond_rates - base_rates)
            / (1 - base_rates)
        )
    return L_vals


def compile_timed_sequence_counts(seq_list, timed_seq_list, states):
    next_counts = []
    cond_counts = []    
    for seq_index in range(len(seq_list)):
        # count_res = get_counts(seq, states)
        # count_res = get_timed_counts(seq_list[seq_index], timed_seq_list[seq_index], states)
        count_res = get_affect_behaviors_counts(seq_list[seq_index], states)
        next_counts.append(count_res[0])
        cond_counts.append(count_res[1])
    return np.array(next_counts), np.array(cond_counts)

def compile_sequence_counts(seq_list, states, consider_affect_behaviors_cooccurence=False):
    next_counts = []
    cond_counts = []    
    for seq_index in range(len(seq_list)):
        # count_res = get_counts(seq, states)
        if consider_affect_behaviors_cooccurence:
          count_res = get_affect_behaviors_cooccurence_counts(seq_list[seq_index], states)
          # print("consider_affect_behaviors_cooccurence", count_res)
        else:
          count_res = get_affect_behaviors_counts(seq_list[seq_index], states)
          
        next_counts.append(count_res[0])
        cond_counts.append(count_res[1])
    return np.array(next_counts), np.array(cond_counts)


def print_vals(val_array, state_dict, title):
    print('\n\n' + title + '\n')
    print('Prev\\Next' + '\t' + '\t'.join(map(str, list(state_dict.values()))))
    for i in range(len(state_dict)):
        print(state_dict[i] + '\t\t'
              + '\t'.join(map(str, val_array[i, :].round(4))))
    return


def run_simulations(
        num_trials=50000,
        seq_length=21,
        states=['A', 'B', 'C', 'D'],
        base_rates=np.array([0.45, 0.45, 0.05, 0.05]),
        verbose=True,
        include_self_transitions=True,
        compute_L=True
):
    num_states = len(states)
    state_dict = {}
    for i in range(num_states):
        state_dict[i] = states[i]
    seq_list = []
    reduced_seq_list = []
    for i in range(num_trials):
        seq = generate_sequence(seq_length, state_dict, base_rates,
                    include_self_transitions=include_self_transitions)        
        reduced_seq = remove_self_transitions(seq)

        seq_list.append(seq)
        reduced_seq_list.append(reduced_seq)

    res = []
    # Compute L_star using individual base rates from full sequences with
    # self-transitions included     
    res.append(
        compute_statistic(seq_list, states, L_star=True,
                          use_mean_rates=False)
    )
    # Compute L_star using mean base rates from full sequences with
    # self-transitions included         
    res.append(
        compute_statistic(seq_list, states, L_star=True,
                          use_mean_rates=True)
    )
    if compute_L:
        # Compute L using individual base rates from reduced sequences with
        # self-transitions removed             
        res.append(
            compute_statistic(seq_list, states, L_star=False,
                              use_mean_rates=False)
        )
        # Compute L_star using mean base rates from full sequences with
        # self-transitions included                  
        res.append(
            compute_statistic(seq_list, states, L_star=False,
                              use_mean_rates=True)
        )            

    
    if verbose:
        title_list = [
            'L_star with individual base rates from full sequences',
            'L_star with mean base rates from full sequences',
            'L with individual base rates from reduced sequences',
            'L with mean base rates from reduced sequences',
        ]
        end_ind = 2
        if compute_L:
            end_ind = len(title_list)
        for i in range(end_ind):
            print_vals(res[i], state_dict, title_list[i])
            
    return (
        seq_list,
        reduced_seq_list,
        res
    )


def compute_statistic(seq_list, states, L_star=True,
                      use_mean_rates=True, consider_affect_behaviors_cooccurence=False):
    """ General function for computing L_star and L statistics

    Parameters
    ----------
    seq_list : list of lists
        Each entry in the list is a sequence (list) of affective states; note 
        that self-transitions are automatically removed if L_star is false
        Example:
            [
                ['A', 'C', 'C', 'B', 'C'],
                ['B', 'C', 'A', 'C'],
                ['C', 'C', 'C', 'B', 'B', 'A']
            ]
    states : list 
        List containing all possible affective states 
        Example:
            ['A', 'B', 'C']
    L_star : bool, default=True 
        If true compute L_star statistic; otherwise, remove 
        self-transitions and compute L statistic
    use_mean_rates : bool, default=True
        If true compute base rates averaged over all sequences; otherwise,
       compute base rates individually per sequence
    """
    
    if L_star:
        input_list = seq_list
    else:
        input_list = []
        for i in range(len(seq_list)):
            input_list.append(remove_behavior_self_transitions(seq_list[i]))
        
    next_counts, cond_counts = compile_sequence_counts(input_list, states, consider_affect_behaviors_cooccurence)

    # print("next_counts", next_counts)
    # print("cond_counts", cond_counts)

    num_states = len(states)
    res = np.full((num_states, num_states), np.nan)
    test = np.full((num_states, num_states), object)
    for i in range(num_states):
        for j in range(num_states):
            if i != j:
                if L_star:
                    aux = get_L_star_vals(i, j, next_counts, cond_counts, use_mean_rates=use_mean_rates)

                    res[i, j] = np.mean(aux)
                    test[i, j] = aux
                else:
                    aux = get_L_vals(i, j, next_counts, cond_counts, use_mean_rates=use_mean_rates)
                    
                    res[i, j] = np.mean(aux)
                    test[i, j] = aux

    return res, test


def base_rate_analysis(
        states=['A', 'B', 'C', 'D'],
        base_rates=np.ones(4)*0.25,
        num_steps=24,
        rate_step=[0.03, -0.01, -0.01, -0.01],
        num_trials=50000,
        seq_length=21
):
    """ Run numerical experiments 
    
    Experiment 1 parameters: 
        states=['A', 'B', 'C', 'D'], 
        base_rates=np.ones(4)*0.25, 
        num_steps=24, 
        rate_step=[0.03, -0.01, -0.01, -0.01],
        num_trials=50000,
        seq_length=21

    Experiment 2 parameters: 
        states=['A', 'B', 'C', 'D'], 
        base_rates=np.ones(4)*0.25, 
        num_steps=23, 
        rate_step=[0.01, 0.01, -0.01, -0.01],
        num_trials=50000,
        seq_length=21
    
    """
    rate_step = np.array(rate_step)
    indiv_rate_results = []
    mean_rate_results = []
    all_base_rates = []
    for i in range(num_steps):
        sim_res = run_simulations(
            num_trials=num_trials,
            seq_length=seq_length,
            states=states,
            base_rates=base_rates,
            verbose=False,
            include_self_transitions=True,
            compute_L=False
        )
        indiv_rate_results.append(sim_res[2][0])
        mean_rate_results.append(sim_res[2][1])
        all_base_rates.append(list(base_rates))
        if i < num_steps - 1:
            base_rates += rate_step
    return indiv_rate_results, mean_rate_results, all_base_rates



Execution 3: Definition of the functions and statistical calculations
---
This part of the code is the definition of the auxiliary functions for the calculation of the L metric, the p-value statistics and multiple tests and for the printing part of the results and the affect dynamics graph.


In [ ]:
# Function that computes the list of students that should be included in the analysis of the dynamics of emotions
# Returns a list with the ids of the students that will be included
# Output example: [1, 2, 3, 4, 5]

# students_amount -> the number of students that should be considered
#   by default this value is 30, but any value can be passed by parameter

# gender_filter -> specify whether the students should be filtered according their gender
#   by default this value is False

# gender_text -> specify which gender "M" or "F" should be used to filter students
#   by default this value is "M"

# show_info -> receives a boolean that informs if it is to display the information of the selected students
#   by default this value is True

def compute_students_to_include(students_amount=30,
                                gender_filter=False,
                                gender_text="M",
                                show_info=True):

  students_to_include = []
  for n_student in range(1, students_amount + 1, 1):
    include_n_student = False
    
    # condition to include or not students according their gender
    if gender_filter:
      if df_students_info[df_students_info.student_id == n_student].gender.values[0] == gender_text:
        include_n_student = True
      else: 
        include_n_student = False

    if not gender_filter:
      include_n_student = True

    if include_n_student:
      students_to_include.append(n_student)

  if show_info:
    print("Number of students: ", len(students_to_include))
    print("Students ids: ", students_to_include)
  
  return students_to_include


# receives a list of students ids to be considered and filter the emotion labels based on this list 
# and filters that can be modified according to the following paramenters

# students_to_include -> a list of students id to be considered
#   by default this value is []

# consider_emotion_duration -> a boolean to define whether the emotion labels dataset with permanence time should be considered
#   by default this value is False

# consider_emotion_duration -> a boolean to define whether the emotion labels should be filtered by the permanence time
#   by default this value is False

# emotion_to_filter_text -> the emotion that will be used to filter as the source emotion
#   by default this value is "ENG"

# duration_greater_than -> a boolean to define whether the permanence time in the emotion_to_filter_text should be greater or less then a certain threshold
#   by default this value is True

# duration_threshold_method -> defines the method to calculate the emotion duration threshold
#   by default this value is "mean"

# show_info -> receives a boolean that informs if it is to display the information of the selected students
#   by default this value is True
def compute_list_of_emotion_labels(students_to_include=[],
                                   consider_emotion_duration=False,
                                   filter_emotion_duration=False,
                                   emotion_to_filter_text="ENG",
                                   duration_greater_than=True,
                                   duration_threshold_method="mean",
                                   show_info=True):

  # it computes the emotion sequences for each student
  transitions_list = []
  duration_threshold = calculate_emotion_duration_stats(emotion_to_filter_text)[duration_threshold_method]
  for n_student in students_to_include:
    if consider_emotion_duration:
      emotions_duration_labels = df_emotions_duration[df_emotions_duration.student_id == n_student]
      if filter_emotion_duration:
        if duration_greater_than:
          emotions_duration_labels = emotions_duration_labels[((emotions_duration_labels.emotion_label == emotion_to_filter_text) & (emotions_duration_labels.duration >= duration_threshold) | (emotions_duration_labels.emotion_label != emotion_to_filter_text))]
        else:
          emotions_duration_labels = emotions_duration_labels[((emotions_duration_labels.emotion_label == emotion_to_filter_text) & (emotions_duration_labels.duration <= duration_threshold) | (emotions_duration_labels.emotion_label != emotion_to_filter_text))]

      transitions_list.append(list(emotions_duration_labels.emotion_label.values))
    else:
      # transitions_list.append(list(df_labels[df_labels.student_id == n_student].emotion_label.values))
      transitions_list.append(list(df_labels[df_labels.student_id == n_student][["emotion_label", "behavior_label"
]].apply(lambda x: [x["emotion_label"], x["behavior_label"]], axis=1)))

  if show_info:
    print("Affective states: ", emotions_list)
    print("Number of transition lists: ", len(transitions_list))
    print("Labels for each list:")
    for i in range(len(transitions_list)):
      print(str(students_to_include[i]) + ": " + str(len(transitions_list[i])))
    if consider_emotion_duration and filter_emotion_duration:
      if duration_greater_than:
        print("Number of resulting transitions: ", len(df_emotions_duration[(df_emotions_duration.emotion_label == emotion_to_filter_text) & (df_emotions_duration.duration >= duration_threshold)]))
      else: 
        print("Number of resulting transitions: ", len(df_emotions_duration[(df_emotions_duration.emotion_label == emotion_to_filter_text) & (df_emotions_duration.duration <= duration_threshold)]))

  return transitions_list


# compute the number of transitions between all the states and students
def get_emotion_transitions_counts(list_seq, states):
  cond_count = {a: {b: 0 for b in states} for a in states}
  for j in np.arange(0, len(list_seq)):
    seq = list_seq[j]
    num_tr = len(seq) - 1
    # Compute next and conditional counts
    for i in np.arange(1, len(seq)):
      for a in states:
          if seq[i - 1][0] == a:
              for b in states:
                  if seq[i][1] == b:
                      cond_count[a][b] += 1
                      break
          if seq[i - 1][1] == a:
              for b in states:
                  if seq[i][0] == b:
                      cond_count[a][b] += 1
                      break

  return cond_count


def get_emotion_behavior_cooccurence_counts(list_seq, states):
  cond_count = {a: {b: 0 for b in states} for a in states}
  for j in np.arange(0, len(list_seq)):
    seq = list_seq[j]
    num_tr = len(seq) - 1
    # Compute next and conditional counts
    for i in np.arange(1, len(seq)):
      for a in states:
          if seq[i - 1][1] == a:
              for b in states:
                  if seq[i - 1][0] == b:
                      cond_count[a][b] += 1
                      break

  return cond_count


# compute and return the statistics about the permanence time of an emotion entered as text
def calculate_emotion_duration_stats(emotion_text):
  emotion_duration_stats = {}
  emotion_duration_stats["min"] = df_emotions_duration.loc[df_emotions_duration.emotion_label == emotion_text].duration.min()
  emotion_duration_stats["max"] = df_emotions_duration.loc[df_emotions_duration.emotion_label == emotion_text].duration.max()
  emotion_duration_stats["mean"] = df_emotions_duration.loc[df_emotions_duration.emotion_label == emotion_text].duration.mean()
  emotion_duration_stats["median"] = df_emotions_duration.loc[df_emotions_duration.emotion_label == emotion_text].duration.median()
  emotion_duration_stats["std"] = df_emotions_duration.loc[df_emotions_duration.emotion_label == emotion_text].duration.std()
  emotion_duration_stats["shapiro"], emotion_duration_stats["shapiro_p_value"] = shapiro(df_emotions_duration.loc[df_emotions_duration.emotion_label == emotion_text].duration)
  return emotion_duration_stats

# print the statistics of an emotion entered as text
def print_emotion_duration_stats(emotion_text):
  emotion_duration_stats = calculate_emotion_duration_stats(emotion_text)
  print("-------------- Emotion ", emotion_text, " ----------------------------")
  print("Min: ", emotion_duration_stats["min"])
  print("Max: ", emotion_duration_stats["max"])
  print("Mean: ", emotion_duration_stats["mean"])
  print("Median: ", emotion_duration_stats["median"])
  print("Std. Dev.: ", emotion_duration_stats["std"])
  print('Shapiro-Wilk=%.3f, p=%.3f' % (emotion_duration_stats["shapiro"], emotion_duration_stats["shapiro_p_value"]))

# receives a matrix with the L metric average for each emotion transition combination and calculates the p-value for each combination
def calculate_p_values(states_list, L_metric_average_results, level_at_change):
  num_states = len(states_list)
  matrix_p_values = np.full((num_states, num_states), np.nan)
  for i in range(num_states):
    for j in range(num_states):
      if i != j:
        matrix_p_values[i][j] = stats.ttest_1samp(list(L_metric_average_results[i][j]), level_at_change).pvalue
  return matrix_p_values

# calculate Benjamini/Hochberg multiple test adjustment for the p-values matrix
# https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html
def calculate_multiple_t_test_adjustment(states_list, matrix_p_values):
  num_states = len(states_list)
  multiple_tests_matrix = np.full((num_states ,num_states), np.nan)
  multiple_tests_reject_matrix = np.full((num_states ,num_states), np.nan)

  for i in range(num_states):
    pvals =[x for x in matrix_p_values[i] if str(x) != 'nan']
    if len(pvals) > 0:
      line_bh = multi.multipletests(pvals, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)
      for j in range(num_states - 1):
        column = j
        if j >= i:
          column = column + 1
        multiple_tests_matrix[i][column] = line_bh[1][j]
        multiple_tests_reject_matrix[i][column] = line_bh[0][j]
  
  return multiple_tests_matrix, multiple_tests_reject_matrix


# print the table with information about the all the emotion's transitions allowing to select which information is going to be printed
def print_vals(l_matrix, p_value_matrix, multiple_tests_matrix, multiple_tests_reject_matrix, count_matrix, states_list, print_types, lines_to_include=[0, 1, 2, 3, 4], columns_to_include=[0, 1, 2, 3, 4]):
  
  labels_to_print = ";".join(label for (index, label) in enumerate(states_list) if index in columns_to_include)
  print('Prev\\Next;' + labels_to_print)
  for i in range(len(states_list)):
    if i in lines_to_include:
      print(states_list[i] + ';', end ="")
      for j in range(len(states_list)):
        if j in columns_to_include:
          if "L_metric" in print_types:
            print('na' if np.isnan(l_matrix[i,j]) else format((l_matrix[i,j]).round(4), '.4f'), end="", flush=True)
          if "p_value" in print_types:
            print(' (', end="", flush=True) if len(print_types) > 1 else False
            print('na' if np.isnan(p_value_matrix[i,j]) else format((p_value_matrix[i,j]).round(4), '.4f'), end="", flush=True)
            print(') ', end="", flush=True) if len(print_types) > 1 else False
          if "multiple_test_p_value" in print_types:
            print(' |', end="", flush=True) if len(print_types) > 1 else False
            print('na' if np.isnan(multiple_tests_matrix[i,j]) else format((multiple_tests_matrix[i,j]).round(4), '.4f'), end="", flush=True)
            print('| ', end="", flush=True) if len(print_types) > 1 else False
          if "transition_count" in print_types:
            print(' [', end="", flush=True) if len(print_types) > 1 else False
            print('na' if np.isnan(count_matrix[states_list[i]][states_list[j]]) else str((count_matrix[states_list[i]][states_list[j]])), end="", flush=True) 
            print('] ', end="", flush=True) if len(print_types) > 1 else False
          if "multiple_test_significance" in print_types:
            print(' {', end="", flush=True) if len(print_types) > 1 else False
            print('na' if np.isnan(multiple_tests_reject_matrix[i,j]) else format((multiple_tests_reject_matrix[i,j]).round(4), '.1f'), end="", flush=True)
            print('} ', end="", flush=True) if len(print_types) > 1 else False
          
          if j < len(states_list) - 1:
            print(';', end="", flush=True)
      print("")
  return

# print the affect dynamics graph based on the L matrix and the multiple tests adjustment rejection status
def print_affect_dynamics_graph(l_matrix, multiple_tests_reject_matrix, states_list, threshold):
  fig, ax = plt.subplots(figsize=(10,10))

  G = nx.MultiDiGraph()  # Create empty graph

  # create the nodes based on the states list
  G.add_nodes_from(states_list)

  for i in range(len(states_list)):
    for j in range(len(states_list)):
      is_significant = (np.zeros(1)[0] if np.isnan(multiple_tests_reject_matrix[i,j]) else multiple_tests_reject_matrix[i,j]).round(1)
      if i != j and is_significant > 0:
        transition_l = (np.zeros(1)[0] if np.isnan(l_matrix[i,j]) else l_matrix[i,j]).round(2)
        transition_color = 'red' if transition_l < threshold else 'blue'
        # add the edges with the l_metric as weight only if the transition is significant (based on the multiple test adjustment)
        # change the color for red if the transition is unlikely and blue if it is more likely to occur
        G.add_edge(states_list[i], states_list[j], color=transition_color, weight=format(transition_l, '.2f'))
  
  pos =  nx.circular_layout(G, scale=1)  # List of positions of nodes

  colors = []

  for (u,v,attrib_dict) in list(G.edges.data()):
      colors.append(attrib_dict['color'])

  weights = [(0.4 + abs(float(d['weight'])) + abs(float(d['weight'])) * 4) for u,v,d in G.edges(data=True)]
  nx.draw(G, pos, with_labels=True, connectionstyle='arc3, rad = 0.15', node_size=1200, edge_color=colors, width=weights, node_color='lightgreen')
  edge_labels = dict([((u,v,),d['weight']) for u,v,d in G.edges(data=True)])

  nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, label_pos=0.2, font_size=11)


Execution 4: Dynamics between affect and behaviors
---
This part of the code executes the previous defined functions and set the paremeters to calculate dynamics between affect and behaviors, considering the data from all the students.


In [ ]:
print("The probability of an affect to onset a behavior - all students")

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=False,
                                                        gender_text="M",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[0, 1, 2, 3, 4],
           columns_to_include=[5, 6, 7, 8, 9])



The probability of an affect to onset a behavior - all students
Number of students:  30
Students ids:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Prev\Next;ON_TASK;ON_SYSTEM;ON_TASK_RES;OFF_TASK;ON_TASK_CONV
ENG;0.2614 (0.0000)  [2150]  {1.0} ;0.1479 (0.0008)  [169]  {1.0} ;0.0437 (0.1142)  [319]  {0.0} ;0.1212 (0.0318)  [121]  {1.0} ;0.0343 (0.0446)  [92]  {0.0} 
CON;0.4013 (0.0000)  [643]  {1.0} ;-0.0208 (0.0012)  [160]  {1.0} ;0.0421 (0.3274)  [45]  {0.0} ;0.0981 (0.3873)  [52]  {0.0} ;0.1364 (0.1208)  [281]  {0.0} 
FRU;0.4691 (0.0067)  [60]  {1.0} ;-0.0170 (0.0126)  [88]  {1.0} ;0.0329 (0.6783)  [1]  {0.0} ;0.0245 (0.4464)  [13]  {0.0} ;0.1329 (0.3421)  [15]  {0.0} 
BOR;0.6077 (0.0000)  [108]  {1.0} ;0.0403 (0.6986)  [115]  {0.0} ;0.0331 (0.5180)  [2]  {0.0} ;0.0557 (0.9000)  [45]  {0.0} ;-0.0414 (0.0000)  [0]  {1.0} 
OTH;0.5788 (0.0000)  [175]  {1.0} ;0.0543 (0.7361)  [372]  {0.0} ;0.0128 (0.0045)  [12]  {1.0} ;0

In [ ]:
print("The probability of an affect to onset a behavior - female students")

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=True,
                                                        gender_text="F",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[0, 1, 2, 3, 4],
           columns_to_include=[5, 6, 7, 8, 9])



The probability of an affect to onset a behavior - female students
Number of students:  16
Students ids:  [1, 2, 6, 9, 11, 13, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27]
Prev\Next;ON_TASK;ON_SYSTEM;ON_TASK_RES;OFF_TASK;ON_TASK_CONV
ENG;0.2557 (0.0001)  [1214]  {1.0} ;0.1259 (0.0499)  [62]  {0.0} ;0.0553 (0.6759)  [161]  {0.0} ;0.1237 (0.0720)  [72]  {0.0} ;0.0439 (0.4171)  [38]  {0.0} 
CON;0.4281 (0.0007)  [262]  {1.0} ;0.0116 (0.1324)  [144]  {0.0} ;0.0133 (0.0288)  [22]  {1.0} ;0.0907 (0.4879)  [42]  {0.0} ;0.1140 (0.4418)  [158]  {0.0} 
FRU;0.2987 (0.2193)  [28]  {0.0} ;-0.0285 (0.0763)  [86]  {0.0} ;0.1098 (0.7621)  [1]  {0.0} ;0.0349 (0.7297)  [8]  {0.0} ;0.2086 (0.2254)  [8]  {0.0} 
BOR;0.6031 (0.0004)  [55]  {1.0} ;0.0236 (0.4901)  [93]  {0.0} ;0.0687 (0.9342)  [2]  {0.0} ;0.0430 (0.6888)  [27]  {0.0} ;-0.0504 (0.0000)  [0]  {1.0} 
OTH;0.5579 (0.0001)  [70]  {1.0} ;0.0356 (0.4681)  [201]  {0.0} ;0.0038 (0.0035)  [6]  {1.0} ;0.0658 (0.9480)  [209]  {0.0} ;0.0241 (0.1414)  [22]  {0.0

In [ ]:
print("The probability of an affect to onset a behavior - male students")

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=True,
                                                        gender_text="M",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[0, 1, 2, 3, 4],
           columns_to_include=[5, 6, 7, 8, 9])



The probability of an affect to onset a behavior - male students
Number of students:  14
Students ids:  [3, 4, 5, 7, 8, 10, 12, 14, 15, 16, 25, 28, 29, 30]
Prev\Next;ON_TASK;ON_SYSTEM;ON_TASK_RES;OFF_TASK;ON_TASK_CONV
ENG;0.2679 (0.0003)  [936]  {1.0} ;0.1731 (0.0075)  [107]  {1.0} ;0.0304 (0.0556)  [158]  {0.0} ;0.1182 (0.2250)  [49]  {0.0} ;0.0233 (0.0142)  [54]  {1.0} 
CON;0.3706 (0.0102)  [381]  {1.0} ;-0.0579 (0.0023)  [16]  {1.0} ;0.0750 (0.7355)  [23]  {0.0} ;0.1065 (0.5723)  [10]  {0.0} ;0.1620 (0.1628)  [123]  {0.0} 
FRU;0.6183 (0.0174)  [32]  {1.0} ;-0.0070 (0.1171)  [2]  {0.0} ;-0.0344 (0.0004)  [0]  {1.0} ;0.0154 (0.5024)  [5]  {0.0} ;0.0666 (0.9668)  [7]  {0.0} 
BOR;0.6147 (0.0096)  [53]  {1.0} ;0.0653 (0.9816)  [22]  {0.0} ;-0.0203 (0.0000)  [0]  {1.0} ;0.0749 (0.9174)  [18]  {0.0} ;-0.0280 (0.0002)  [0]  {1.0} 
OTH;0.6046 (0.0000)  [105]  {1.0} ;0.0773 (0.6410)  [171]  {0.0} ;0.0240 (0.2197)  [6]  {0.0} ;-0.0169 (0.0022)  [216]  {1.0} ;-0.0011 (0.0000)  [3]  {1.0} 



Execution 5: Dynamics between behaviors and affect
---
This part of the code executes the previous defined functions and set the paremeters to calculate dynamics between behaviors and affect, considering the data from all the students.


In [ ]:
print("The probability of a behavior to onset an affect - all students")

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=False,
                                                        gender_text="M",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[5, 6, 7, 8, 9],
           columns_to_include=[0, 1, 2, 3, 4])


The probability of a behavior to onset an affect - all students
Number of students:  30
Students ids:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Prev\Next;ENG;CON;FRU;BOR;OTH
ON_TASK;0.1620 (0.0011)  [2130]  {1.0} ;0.1700 (0.0000)  [664]  {1.0} ;0.0343 (0.0059)  [69]  {1.0} ;0.0238 (0.0002)  [98]  {1.0} ;0.1996 (0.0000)  [183]  {1.0} ;
ON_SYSTEM;0.6471 (0.0000)  [170]  {1.0} ;-0.0535 (0.0008)  [158]  {1.0} ;0.0004 (0.0001)  [88]  {1.0} ;0.0193 (0.0347)  [120]  {1.0} ;0.0411 (0.5503)  [366]  {0.0} ;
ON_TASK_RES;0.6015 (0.0001)  [328]  {1.0} ;0.1304 (0.4225)  [42]  {0.0} ;-0.0158 (0.0000)  [0]  {1.0} ;-0.0173 (0.0006)  [1]  {1.0} ;-0.0198 (0.0619)  [9]  {0.0} ;
OFF_TASK;0.4471 (0.0000)  [115]  {1.0} ;0.0595 (0.9462)  [49]  {0.0} ;-0.0204 (0.0000)  [8]  {1.0} ;0.0756 (0.6455)  [52]  {0.0} ;0.0681 (0.9145)  [428]  {0.0} ;
ON_TASK_CONV;0.4118 (0.0021)  [102]  {1.0} ;0.0861 (0.6310)  [267]  {0.0} ;0.0305 (0.3427)  [12]  {0

In [ ]:
print("The probability of a behavior to onset an affect - female students")

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=True,
                                                        gender_text="F",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[5, 6, 7, 8, 9],
           columns_to_include=[0, 1, 2, 3, 4])

The probability of a behavior to onset an affect - female students
Number of students:  16
Students ids:  [1, 2, 6, 9, 11, 13, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27]
Prev\Next;ENG;CON;FRU;BOR;OTH
ON_TASK;0.1564 (0.0073)  [1204]  {1.0} ;0.1720 (0.0005)  [268]  {1.0} ;0.0354 (0.0857)  [35]  {0.0} ;0.0244 (0.0026)  [47]  {1.0} ;0.1984 (0.0003)  [80]  {1.0} ;
ON_SYSTEM;0.5936 (0.0001)  [60]  {1.0} ;-0.0280 (0.0897)  [141]  {0.0} ;-0.0191 (0.0000)  [85]  {1.0} ;0.0428 (0.5594)  [98]  {0.0} ;0.0607 (0.9695)  [202]  {0.0} ;
ON_TASK_RES;0.6784 (0.0010)  [162]  {1.0} ;0.0501 (0.8647)  [23]  {0.0} ;-0.0147 (0.0000)  [0]  {1.0} ;-0.0079 (0.0803)  [1]  {0.0} ;-0.0226 (0.1468)  [6]  {0.0} ;
OFF_TASK;0.5286 (0.0001)  [75]  {1.0} ;0.1245 (0.3693)  [42]  {0.0} ;-0.0204 (0.0000)  [5]  {1.0} ;0.0589 (0.9032)  [31]  {0.0} ;-0.0439 (0.0002)  [201]  {1.0} ;
ON_TASK_CONV;0.4345 (0.0085)  [43]  {1.0} ;0.0957 (0.6430)  [152]  {0.0} ;0.0182 (0.1146)  [6]  {0.0} ;-0.0296 (0.0000)  [1]  {1.0} ;0.1016 (0.5746)  

In [ ]:
print("The probability of a behavior to onset an affect - male students")

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=True,
                                                        gender_text="M",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[5, 6, 7, 8, 9],
           columns_to_include=[0, 1, 2, 3, 4])

The probability of a behavior to onset an affect - male students
Number of students:  14
Students ids:  [3, 4, 5, 7, 8, 10, 12, 14, 15, 16, 25, 28, 29, 30]
Prev\Next;ENG;CON;FRU;BOR;OTH
ON_TASK;0.1685 (0.0483)  [926]  {1.0} ;0.1678 (0.0108)  [396]  {1.0} ;0.0330 (0.0294)  [34]  {1.0} ;0.0231 (0.0292)  [51]  {1.0} ;0.2010 (0.0035)  [103]  {1.0} ;
ON_SYSTEM;0.7130 (0.0000)  [110]  {1.0} ;-0.0849 (0.0005)  [17]  {1.0} ;0.0244 (0.2006)  [3]  {0.0} ;-0.0096 (0.0001)  [22]  {1.0} ;0.0170 (0.4272)  [164]  {0.0} ;
ON_TASK_RES;0.5160 (0.0307)  [166]  {1.0} ;0.2197 (0.3437)  [19]  {0.0} ;-0.0171 (0.0000)  [0]  {1.0} ;-0.0277 (0.0001)  [0]  {1.0} ;-0.0168 (0.2730)  [3]  {0.0} ;
OFF_TASK;0.3531 (0.0355)  [40]  {0.0} ;-0.0156 (0.1546)  [7]  {0.0} ;-0.0205 (0.0000)  [3]  {1.0} ;0.0950 (0.5389)  [21]  {0.0} ;0.1972 (0.1941)  [227]  {0.0} ;
ON_TASK_CONV;0.3795 (0.1267)  [59]  {0.0} ;0.0723 (0.8907)  [115]  {0.0} ;0.0482 (0.8532)  [6]  {0.0} ;-0.0208 (0.0000)  [0]  {1.0} ;0.1422 (0.6407)  [5]  {0.0} ;



Execution 6: Co-occurrence between behaviors and affect
---
This part of the code executes the previous defined functions and set the paremeters to calculate the co-occurrence between behaviors and affect, considering the data from all the students.


In [ ]:
print("The probability of co-occurrence betwen behaviors and affects - all students")

# variable to define whether the coocurrence calculation is enabled
consider_affect_behaviors_cooccurence=True

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=False,
                                                        gender_text="M",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False,
                                                               consider_affect_behaviors_cooccurence=consider_affect_behaviors_cooccurence)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[5, 6, 7, 8, 9],
           columns_to_include=[0, 1, 2, 3, 4])

The probability of co-occurrence betwen behaviors and affects - all students
Number of students:  30
Students ids:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Prev\Next;ENG;CON;FRU;BOR;OTH
ON_TASK;0.6402 (0.0000)  [2130]  {1.0} ;0.2422 (0.0000)  [664]  {1.0} ;0.0482 (0.2367)  [69]  {0.0} ;0.0361 (0.0079)  [98]  {1.0} ;0.0333 (0.0034)  [183]  {1.0} ;
ON_SYSTEM;0.1845 (0.0213)  [170]  {1.0} ;0.0549 (0.7645)  [158]  {0.0} ;0.0211 (0.0036)  [88]  {1.0} ;0.1533 (0.0116)  [120]  {1.0} ;0.5863 (0.0000)  [366]  {1.0} ;
ON_TASK_RES;0.8187 (0.0000)  [328]  {1.0} ;0.1401 (0.1152)  [42]  {0.0} ;0.0000 (0.0000)  [0]  {1.0} ;0.0175 (0.0196)  [1]  {1.0} ;0.0237 (0.0292)  [9]  {1.0} ;
OFF_TASK;0.0811 (0.5065)  [115]  {0.0} ;0.0718 (0.8090)  [49]  {0.0} ;0.0234 (0.0061)  [8]  {1.0} ;0.0662 (0.8561)  [52]  {0.0} ;0.7575 (0.0000)  [428]  {1.0} ;
ON_TASK_CONV;0.1477 (0.1068)  [102]  {0.0} ;0.6628 (0.0000)  [267]  {1.0} ;0.1053 (0.3788)  [12]  {0.0} ;0.0000 (0.0000)  [1]  {1.0} ;0.0842 (0.5652)  [28]  {0.0} ;


In [ ]:
print("The probability of co-occurrence betwen behaviors and affects - female students")

# variable to define whether the coocurrence calculation is enabled
consider_affect_behaviors_cooccurence=True

# it computes the list of students to be included in the analysis 
students_to_include_list = compute_students_to_include( students_amount=30,
                                                        gender_filter=True,
                                                        gender_text="F",
                                                        show_info=True)

# it computes the list of emotions labels to be considered according to the list of students
transitions_list = compute_list_of_emotion_labels(students_to_include=students_to_include_list,
                                                  consider_emotion_duration=False,
                                                  filter_emotion_duration=False,
                                                  emotion_to_filter_text="ENG",
                                                  duration_greater_than=True,
                                                  duration_threshold_method="mean",
                                                  show_info=False)

# based on the list of emotions labels, it computes the L metric matrix and the averaged L matrix
L_metric_results, L_metric_average_results = compute_statistic(seq_list=transitions_list,
                                                               states=emotions_behaviors_list, 
                                                               L_star=False, 
                                                               use_mean_rates=False,
                                                               consider_affect_behaviors_cooccurence=consider_affect_behaviors_cooccurence)

# compute the p values for each transition according to the L matrix and the average L matrix
p_values_matrix = calculate_p_values(states_list=emotions_behaviors_list, 
                                     L_metric_average_results=L_metric_average_results, 
                                     level_at_change=L_metric_at_chance_threshold)

# compute the t test adjustment according to the BH post hoc method 
multiple_tests_matrix, multiple_tests_reject_matrix = calculate_multiple_t_test_adjustment(states_list=emotions_behaviors_list, 
                                                                                           matrix_p_values=p_values_matrix)

# compute the emotion transitions count
emotion_counts = get_emotion_transitions_counts(list_seq=transitions_list, 
                                                states=emotions_behaviors_list)


# print the results according to the predefined metrics
print_vals(L_metric_results, 
           p_values_matrix, 
           multiple_tests_matrix, 
           multiple_tests_reject_matrix, 
           emotion_counts, 
           emotions_behaviors_list,
           print_types=table_metrics_to_print,
           lines_to_include=[5, 6, 7, 8, 9],
           columns_to_include=[0, 1, 2, 3, 4])

The probability of co-occurrence betwen behaviors and affects - female students
Number of students:  16
Students ids:  [1, 2, 6, 9, 11, 13, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27]
Prev\Next;ENG;CON;FRU;BOR;OTH
ON_TASK;0.6627 (0.0000)  [1204]  {1.0} ;0.2421

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


 (0.0001)  [268]  {1.0} ;0.0427 (0.2100)  [35]  {0.0} ;0.0320 (0.0362)  [47]  {1.0} ;0.0205 (0.0021)  [80]  {1.0} ;
ON_SYSTEM;0.1031 (0.4161)  [60]  {0.0} ;0.0833 (0.6404)  [141]  {0.0} ;0.0312 (0.1881)  [85]  {0.0} ;0.2229 (0.0047)  [98]  {1.0} ;0.5594 (0.0000)  [202]  {1.0} ;
ON_TASK_RES;0.8517 (0.0000)  [162]  {1.0} ;0.0700 (0.8479)  [23]  {0.0} ;0.0000 (0.0000)  [0]  {1.0} ;0.0333 (0.4043)  [1]  {0.0} ;0.0450 (0.5769)  [6]  {0.0} ;
OFF_TASK;0.0789 (0.7075)  [75]  {0.0} ;0.1341 (0.3128)  [42]  {0.0} ;0.0296 (0.1734)  [5]  {0.0} ;0.0716 (0.7580)  [31]  {0.0} ;0.6857 (0.0000)  [201]  {1.0} ;
ON_TASK_CONV;0.1235 (0.3488)  [43]  {0.0} ;0.7168 (0.0000)  [152]  {1.0} ;0.0791 (0.7532)  [6]  {0.0} ;0.0000 (0.0000)  [1]  {1.0} ;0.0806 (0.6782)  [23]  {0.0} ;
